In [1]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/10/25 17:26:41] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=380770;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=650656;file://c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\gusta\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
cliente = spark.table("prd.dl_redeok_checkok.tb_cliente")
cliente.show()


+------+--------------------+---------+--------------------+--------------------+--------------------+----+-----------+--------------+---------------+--------------------+----+--------+-------+--------------------+------+-----+-------+--------------------+--------------------+---------+--------------+-----------+--------+-------------+-------+----------+-------+----------+------------+--------------------+--------+---------+----------+----------+-----------+------------+-----------+----------+--------------+-----------+------------+--------------------+---------+-------+---------+--------+----------+---------------+--------+---------+-----------+-------+--------------------+---------------+----------------+----------------+-------------+-------------+-------------+----------------+--------------------+--------------+---------------------+----------------------+------+--------+---------------------+--------------------+---------------------------+--------------------+-------------+-----

In [ ]:
df_filtrado = cliente.filter(cliente["cgcmf"] == "24632614000154")
df_filtrado.show(1000000)

In [3]:
revenda = spark.table("prd.dl_redeok_checkok.tb_revenda")
revenda.show()

+------+---------+-------+--------------------+----------+------------+---------------+-----------+--------------+--------+-------------+---------+--------+--------+--------------+
|CODIGO|DESCRICAO|REPASSE|          DTA_ULTATU|USR_ULTATU|COD_CONTRATO|DTA_ASSCONTRATO|TIP_REVENDA|COD_TABREPASSE|SG_GRUPO|VL_PERCENTUAL|SG_VISREP|SG_DEMCO|CD_CONTA|date_partition|
+------+---------+-------+--------------------+----------+------------+---------------+-----------+--------------+--------+-------------+---------+--------+--------+--------------+
|101549|    IVAN |      N|2010-08-25 11:23:...|  luzinete|        NULL|           NULL|          2|             1|    NULL|         NULL|        S|       N|    NULL|    2025-02-10|
| 95854|    JOAO |      N|2010-09-14 16:53:...|  luzinete|        NULL|           NULL|          2|             1|    NULL|         NULL|        S|       N|    NULL|    2025-02-10|
|259088|    OKABC|      S|2010-10-15 17:17:...|  luzinete|        NULL|           NULL|        

In [4]:
revenda = spark.table("prd.dl_redeok_checkok.vw_receber")
revenda.show()

+-------+----------+-------+------+-------------------+-------------------+-------------------+--------+-------+-------+-----+-------------------+----------+------------+--------------------+--------+----+------------+----------------+--------+-----+----------+---------+----------+---------+-------+---+----+--------+------+-------+--------+----------+----------+----------+-----------+-----------+-----------+-----------+--------------------+--------+---------+-------------+-----------+-------------+---------------+--------+---------+------+--------+-------+----------+---------+---------+-------+---------+-----------+-----------------+--------------+-------+--------------+-------+-------+
| numero|    codigo|cliente| valor|            emissao|         vencimento|          pagto_ate|desconto|receita|especie|banco|         data_pagto|valor_pago|cod_desconto|          observacao|situacao|tipo|cancelamento|mot_cancelamento|num_vias|baixa|quantidade|cadastral|vlr_renpac|vlr_dspac|repasse|mes|

In [16]:
query=  """
SELECT 
    cli.*,
    rev.*,
    rec.*
FROM 
    prd.dl_redeok_checkok.tb_cliente cli
INNER JOIN prd.dl_redeok_checkok.vw_receber rec 
    ON cli.codigo = rec.cliente
INNER JOIN prd.dl_redeok_checkok.tb_revenda rev 
    ON cli.codigo = rev.codigo;



"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
sdf_final_z= spark.sql(query)


In [17]:
sdf_final_z.show()

+------+--------------------+---------+--------------------+--------------------+----------+----+-----------+--------------+---------------+--------------------+----+--------+------+--------------------+------+-----+-------+--------------------+--------------------+---------+--------------+-----------+--------+-------------+-------+----------+-------+----------+------------+--------------------+--------+---------+----------+----------+-----------+------------+-----------+----------+--------------+-----------+------------+--------------------+---------+-------+---------+--------+----------+---------------+--------+---------+-----------+-------+---------------+---------------+----------------+----------------+-------------+-------------+-------------+----------------+-------------------+--------------+---------------------+----------------------+------+--------+---------------------+-------------------+---------------------------+-------------------+-------------+--------------+-------+-

In [5]:
query=  """WITH 
clientes AS 
(
    SELECT 
          COUNT(1)              AS contratos
        , cgcmf                 AS documento
        , MIN(data_cad)         AS data_inicio
        , MAX(bloqueio)         AS data_final
    FROM 
    (
        SELECT DISTINCT
            matriz, codigo, fis_jurid, cgcmf, raz_social, max_conexao, bloqueio,
            sit_cliente, data_cad, vendedor, vlrfatmin, vlr_dispacesso, receita
        FROM 
            prd.dl_redeok_checkok.tb_cliente cli
        WHERE 
            cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14
            AND cli.cgcmf RLIKE '^[0-9]+$'
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE cli.codigo = rev.codigo
            )
    ) 
    GROUP BY cgcmf
),
titulo_cobranca AS 
(
    SELECT 
          documento         AS documento
        , COUNT(meses)      AS faturas
    FROM 
    (
        SELECT DISTINCT
            cli.cgcmf                           AS documento
            , DATE_TRUNC('month', rec.emissao)  AS meses
        FROM 
            prd.dl_redeok_checkok.vw_receber rec
        INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
            ON cli.codigo = rec.cliente
        WHERE 
            rec.situacao IN ('A', 'B')
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE rev.codigo = rec.cliente
            )
            AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14    
            AND cli.cgcmf RLIKE '^[0-9]+$'
    ) 
    GROUP BY 
        documento
),
ultimo_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MAX(rec.emissao) AS ultima_emissao
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
)
SELECT 
      cli.contratos
    , cli.documento
    , cli.data_inicio
    , cli.data_final
    , CASE 
          WHEN cli.data_final IS NULL THEN NULL
          ELSE ult.ultima_emissao
      END AS data_ultimo_boleto
      , tit.faturas
    , ROUND(MONTHS_BETWEEN(COALESCE(cli.data_final, CURRENT_DATE()), cli.data_inicio)) AS meses_contrato 
FROM 
    clientes cli
INNER JOIN titulo_cobranca tit 
    ON cli.documento = tit.documento
LEFT JOIN ultimo_boleto ult
    ON cli.documento = ult.documento;



"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
sdf_final= spark.sql(query)


In [13]:
query_1=  """WITH clientes AS 
(
    SELECT 
          COUNT(1)              AS contratos
        , cgcmf                 AS documento
        , MIN(data_cad)         AS data_inicio
        , MAX(bloqueio)         AS data_final
    FROM 
    (
        SELECT DISTINCT
            matriz, codigo, fis_jurid, cgcmf, raz_social, max_conexao, bloqueio,
            sit_cliente, data_cad, vendedor, vlrfatmin, vlr_dispacesso, receita
        FROM 
            prd.dl_redeok_checkok.tb_cliente cli
        WHERE 
            cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14
            AND cli.cgcmf RLIKE '^[0-9]+$'
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE cli.codigo = rev.codigo
            )
    ) 
    GROUP BY cgcmf
),
titulo_cobranca AS 
(
    SELECT 
          documento         AS documento
        , COUNT(meses)      AS faturas
    FROM 
    (
        SELECT DISTINCT
            cli.cgcmf                           AS documento
            , DATE_TRUNC('month', rec.emissao)  AS meses
        FROM 
            prd.dl_redeok_checkok.vw_receber rec
        INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
            ON cli.codigo = rec.cliente
        WHERE 
            rec.situacao IN ('A', 'B')
            AND NOT EXISTS 
            (
                SELECT 1 
                FROM prd.dl_redeok_checkok.tb_revenda rev 
                WHERE rev.codigo = rec.cliente
            )
            AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
            AND (cli.matriz = cli.codigo OR falsa = 's')
            AND cli.receita IN (9, 10, 46, 58)
            AND cli.cgcmf IS NOT NULL
            AND cli.cgcmf != '23236392000198'
            AND LENGTH(TRIM(cli.cgcmf)) = 14    
            AND cli.cgcmf RLIKE '^[0-9]+$'
    ) 
    GROUP BY 
        documento
),
ultimo_boleto AS 
(
    SELECT 
        cli.cgcmf AS documento,
        MAX(rec.emissao) AS ultima_emissao
    FROM 
        prd.dl_redeok_checkok.vw_receber rec
    INNER JOIN prd.dl_redeok_checkok.tb_cliente cli 
        ON cli.codigo = rec.cliente
    WHERE 
        rec.situacao IN ('A', 'B')
        AND NOT EXISTS 
        (
            SELECT 1 
            FROM prd.dl_redeok_checkok.tb_revenda rev 
            WHERE rev.codigo = rec.cliente
        )
        AND cli.sit_cliente IN ('A', 'F', 'N', 'P', 'D')
        AND (cli.matriz = cli.codigo OR falsa = 's')
        AND cli.receita IN (9, 10, 46, 58)
        AND cli.cgcmf IS NOT NULL
        AND cli.cgcmf != '23236392000198'
        AND LENGTH(TRIM(cli.cgcmf)) = 14    
        AND cli.cgcmf RLIKE '^[0-9]+$'
    GROUP BY cli.cgcmf
    HAVING MAX(rec.emissao) IS NOT NULL  -- Garante que há boletos válidos
)
SELECT 
      cli.contratos
    , cli.documento
    , cli.data_inicio
    , cli.data_final
    , COALESCE(ult.ultima_emissao, NULL) AS data_ultimo_boleto -- Simplificação
    , tit.faturas
    , ROUND(MONTHS_BETWEEN(COALESCE(cli.data_final, CURRENT_DATE()), cli.data_inicio)) AS meses_contrato 
FROM 
    clientes cli
INNER JOIN titulo_cobranca tit 
    ON cli.documento = tit.documento
LEFT JOIN ultimo_boleto ult
    ON cli.documento = ult.documento;




"""


# Executando a consulta e obtendo os dados em um DataFrame Spark
sdf_final_1= spark.sql(query_1)


In [ ]:
from pyspark.sql.functions import col

sdf_final_1 = sdf_final_1.orderBy(col("contratos").desc())
sdf_final_1.show(10000000)
sdf_final_1.count()


In [21]:
df_filtrado = sdf_final_z.filter(sdf_final_z["cgcmf"] == "24632614000154")
df_filtrado.show(1000000)

+------+--------+---------+----+----------+----------+---+---+-----+---------+----------+----+--------+-----+----------+------+-----+-------+--------+------------+---------+--------------+-----------+--------+-------------+-------+----------+-------+----------+------------+--------+--------+---------+----------+----------+-----------+------------+-----------+----------+--------------+-----------+------------+--------+---------+-------+---------+--------+----------+---------------+--------+---------+-----------+-------+---------------+---------------+----------------+----------------+-------------+-------------+-------------+----------------+-------------------+--------------+---------------------+----------------------+------+--------+---------------------+-------------------+---------------------------+-------------------+-------------+--------------+-------+-------+----+------+---------+-------+----------+----------+------------+---------------+-----------+--------------+--------+---

## PARTE 1


- count(1) as contratos → Conta o número de registros distintos para cada cgcmf, representando a quantidade de contratos do cliente.
- cgcmf as documento → Mantém o CNPJ como identificador do cliente.
- min(data_cad) as data_inicio → Determina a data mais antiga de cadastro do cliente.
- max(bloqueio) as data_final → Determina a última data de bloqueio registrada.
- O agrupamento (group by cgcmf) garante que os cálculos de contagem e datas sejam feitos para cada CNPJ.


- A query filtra e limpa os dados da tabela tb_cliente para incluir apenas clientes relevantes, removendo revendedores e garantindo que os CNPJs sejam válidos.
- Após aplicar os filtros, os dados são agrupados por cgcmf, calculando:
- A quantidade de contratos (count(1))
- A data do primeiro contrato (min(data_cad))
- A data do último bloqueio (max(bloqueio))
- O resultado é uma lista de clientes com CNPJ válido e suas respectivas informações contratuais.

## Parte 2

In [6]:
from pyspark.sql.functions import col

sdf_final = sdf_final.orderBy(col("contratos").desc())
sdf_final.show()
sdf_final.count()


+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+
|contratos|     documento|         data_inicio|          data_final|  data_ultimo_boleto|faturas|meses_contrato|
+---------+--------------+--------------------+--------------------+--------------------+-------+--------------+
|       84|08838057000176|2014-04-23 00:00:...|2020-02-19 17:02:...|2024-11-01 00:00:...|     84|          70.0|
|       26|12531484000184|2010-10-27 00:00:...|2023-02-03 00:00:...|2016-04-01 00:00:...|      2|         147.0|
|       20|75758089000159|2007-08-31 00:00:...|2019-09-04 00:00:...|2012-12-10 00:00:...|      1|         144.0|
|       18|09967852000127|1999-12-16 00:00:...|2015-01-27 00:00:...| 2008-11-01 00:00:00|     33|         181.0|
|       16|03461819000144|2014-02-04 15:19:...|2014-07-14 11:23:...| 2014-07-01 00:00:00|      4|           5.0|
|       14|12448302000106|2013-10-24 10:14:...|2020-02-18 11:37:...|2022-06-01 00:00:...|     67

134316

- existem clientes com mais de um contratos , se fosse verificar como um cliente tme uma data de iniciio e a data fim muito depois do ultimo boleto

In [7]:
sdf_final = sdf_final.withColumn("faturas_anos", col("faturas") / 12)
sdf_final = sdf_final.withColumn("meses_contrato_anos", col("meses_contrato") / 12)

In [12]:
# Filtrando as contas onde data_final é nula
sdf_contas_nulas = sdf_final.filter(col("data_final").isNull())

# Exibindo os resultados
sdf_contas_nulas.show()

+---------+--------------+--------------------+----------+------------------+-------+--------------+-------------------+-------------------+
|contratos|     documento|         data_inicio|data_final|data_ultimo_boleto|faturas|meses_contrato|       faturas_anos|meses_contrato_anos|
+---------+--------------+--------------------+----------+------------------+-------+--------------+-------------------+-------------------+
|        2|25381131000197|2014-04-15 00:00:...|      NULL|              NULL|    130|         130.0| 10.833333333333334| 10.833333333333334|
|        2|55773964000164|2004-07-30 10:18:...|      NULL|              NULL|    226|         246.0| 18.833333333333332|               20.5|
|        2|00845661000118|2005-04-08 17:55:...|      NULL|              NULL|    234|         238.0|               19.5| 19.833333333333332|
|        2|14878722000158|2013-08-29 15:19:...|      NULL|              NULL|    105|         137.0|               8.75| 11.416666666666666|
|        2|00

In [8]:
from pyspark.sql.functions import col

num_clientes = sdf_final.count()

print(f"Quantidade de clientes : {num_clientes}")

# Contar valores nulos
num_nulos = sdf_final.filter(col("data_final").isNull()).count()

# Contar valores não nulos
num_nao_nulos = sdf_final.filter(col("data_final").isNotNull()).count()

print(f"Quantidade de clientes ativos: {num_nulos}")
print(f"Quantidade de clientes Finalizados: {num_nao_nulos}")

# Filtrando os registros onde 'meses_contrato_anos' é negativo
clientes_com_meses_negativos = sdf_final.filter(col("meses_contrato_anos") < 0)

# Contando a quantidade de clientes
quantidade_clientes = clientes_com_meses_negativos.count()

# Exibindo o resultado
print(f"Quantidade de clientes com 'meses_contrato_anos' negativo: {quantidade_clientes}")


# Contar quantos registros têm data_final menor que data_inicio
quantidade_datas_invalidas = sdf_final.filter(col("data_final") < col("data_inicio")).count()

print(f"Quantidade de clientes com 'data_final' menor que 'data_inicio': {quantidade_datas_invalidas}")


Quantidade de clientes : 134316
Quantidade de clientes ativos: 13334
Quantidade de clientes Finalizados: 120982
Quantidade de clientes com 'meses_contrato_anos' negativo: 1168
Quantidade de clientes com 'data_final' menor que 'data_inicio': 1177


In [9]:
# Filtrar e manter apenas registros com 'meses_contrato_anos' maior que zero
sdf_final = sdf_final.filter(col("meses_contrato_anos") >= 0)

# Contar a quantidade de clientes após o filtro
num_clientes_filtrados = sdf_final.count()
print(f"Quantidade de clientes após remoção de 'meses_contrato_anos' menores ou iguais a zero: {num_clientes_filtrados}")

Quantidade de clientes após remoção de 'meses_contrato_anos' menores ou iguais a zero: 133148


## Churn Simplificado

In [10]:
# Contagem total de contratos
total_clientes = sdf_final.count()

# Contagem de churn (contratos cancelados)
churn_count = sdf_final.filter(F.col("data_ultimo_boleto").isNotNull()).count()
# Contagem de churn (contratos cancelados)
churn_count_ativos = sdf_final.filter(F.col("data_ultimo_boleto").isNull()).count()

# Cálculo do churn rate
churn_rate = (churn_count / total_clientes) * 100

# Exibir resultado
print(f"Total de clientes: {total_clientes}")
print(f"Clientes cancelados: {churn_count}")
print(f"Clientes Ativos: {churn_count_ativos}")
print(f"Taxa de Churn: {churn_rate:.2f}%")

Total de clientes: 133148
Clientes cancelados: 119814
Clientes Ativos: 13334
Taxa de Churn: 89.99%


In [11]:
from pyspark.sql import functions as F

# Criar a coluna do ano de início do contrato
sdf_churn_anual = sdf_final.withColumn("ano_inicio", F.year("data_inicio")) \
    .groupBy("ano_inicio") \
    .agg(
        F.count("*").alias("total_clientes"),
        F.sum(F.when(F.col("data_ultimo_boleto").isNotNull(), 1).otherwise(0)).alias("contratos_cancelados")
    ) \
    .withColumn("taxa_churn", (F.col("contratos_cancelados") / F.col("total_clientes")) * 100) \
    .orderBy("ano_inicio")

# Converter para Pandas para plotar
df_churn_anual = sdf_churn_anual.toPandas()

# Criar gráfico interativo com Plotly
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_churn_anual["ano_inicio"],
    y=df_churn_anual["taxa_churn"],
    marker_color="red",
    name="Taxa de Churn (%)"
))

# Ajustes de layout
fig.update_layout(
    title="Taxa de Churn Anual",
    xaxis_title="Ano de Início do Contrato",
    yaxis_title="Taxa de Churn (%)",
    template="plotly_white",
    xaxis=dict(tickmode="linear")
)

# Exibir o gráfico
fig.show()
